In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!git clone https://github.com/tkipf/pygcn.git

Cloning into 'pygcn'...
remote: Enumerating objects: 78, done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 78
Receiving objects: 100% (78/78), 226.61 KiB | 1.33 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
%cd pygcn/pygcn

/content/pygcn/pygcn


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/da324dataminingproject2.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/pygcn/pygcn')

In [ ]:
!python train.py

Data Loaded
Epoch: 0001 loss_train: 161729392.0000 acc_train: 0.1011 time: 1.1290s
Epoch: 0002 loss_train: 112011696.0000 acc_train: 0.1013 time: 0.0254s
Epoch: 0003 loss_train: 103070152.0000 acc_train: 0.1878 time: 0.0252s
Epoch: 0004 loss_train: 94600680.0000 acc_train: 0.1878 time: 0.0252s
Epoch: 0005 loss_train: 86785360.0000 acc_train: 0.0962 time: 0.0252s
Epoch: 0006 loss_train: 86306672.0000 acc_train: 0.0878 time: 0.0246s
Epoch: 0007 loss_train: 75623744.0000 acc_train: 0.0951 time: 0.0188s
Epoch: 0008 loss_train: 65385756.0000 acc_train: 0.0919 time: 0.0187s
Epoch: 0009 loss_train: 66157516.0000 acc_train: 0.0987 time: 0.0187s
Epoch: 0010 loss_train: 60455344.0000 acc_train: 0.0991 time: 0.0187s
Epoch: 0011 loss_train: 50941644.0000 acc_train: 0.1292 time: 0.0187s
Epoch: 0012 loss_train: 49830388.0000 acc_train: 0.1872 time: 0.0188s
Epoch: 0013 loss_train: 48188404.0000 acc_train: 0.0871 time: 0.0187s
Epoch: 0014 loss_train: 40668288.0000 acc_train: 0.1859 time: 0.0187s
Epoch

In [ ]:
import pandas as pd
otpt = pd.read_csv("output.csv").drop(columns = ["Unnamed: 0"])

In [ ]:
np.exp(otpt).sum(axis = 1)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
11947    1.0
11948    1.0
11949    1.0
11950    1.0
11951    1.0
Length: 11952, dtype: float64

In [ ]:
pd.DataFrame(otpt.apply(lambda row: row.idxmax(), axis=1))

,0
0,5
1,9
2,4
3,7
4,5
...,...
11947,0
11948,4
11949,4
11950,5


In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def df_to_sparse_matrix(df):
  """
  Converts a pandas dataframe to a sparse matrix in sp.coo_ format.

  Args:
      df: A pandas dataframe.

  Returns:
      A sparse matrix in sp.coo_ format.
  """
  # Get the number of rows and columns
  num_rows = df.shape[0]
  num_cols = df.shape[1]

  # Create a list of tuples for the sparse matrix
  data = []
  for i in range(num_rows):
    for j in range(num_cols):
      if df.iloc[i, j] != 0:
        data.append((i, j, df.iloc[i, j]))

  # Create the sparse matrix
  sparse_matrix = sp.coo_matrix((data, (num_rows, num_cols)))

  return sparse_matrix


def load_data(path="../data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""

    features = pd.read_excel("attributes.xlsx")
    labels = encode_onehot(pd.read_csv("submission.csv")['LABEL'])
    adj = df_to_sparse_matrix(pd.read_csv("adjacency.csv"))

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(10952)
    idx_test = range(10952, 11953)

    features = torch.FloatTensor(np.array(features))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_test


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


In [ ]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from utils import load_data, accuracy
from models import GCN
import pandas as pd

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Load data
adj, features, labels, idx_train, idx_test = load_data()

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    pd.DataFrame(output).to_csv('output.csv')


# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()
